# Query CMIP6 ocean vert coords - native/CMIP6_CVs vs output

In [1]:
%%time
import glob
import json
import os
os.sys.path.insert(0, "/p/user_pub/xclim/persist/software/xsearch/")
import xcdat as xc
import xsearch as xs

import logging
logging.getLogger("xcdat").setLevel(logging.ERROR)

CPU times: user 5.99 s, sys: 1.71 s, total: 7.7 s
Wall time: 5.89 s


## read CMIP6_source_id.json

In [2]:
%%time
path = "/home/durack1/git/CMIP6_CVs"
file = "CMIP6_source_id.json"
with open(os.path.join(path, file), 'r') as f:
    js = json.load(f)
js.keys()
srcIds = js["source_id"]
srcIds.keys()

CPU times: user 828 µs, sys: 2.66 ms, total: 3.49 ms
Wall time: 2.6 ms


dict_keys(['4AOP-v1-5', 'ACCESS-CM2', 'ACCESS-ESM1-5', 'ACCESS-OM2', 'ACCESS-OM2-025', 'ARTS-2-3', 'AWI-CM-1-1-HR', 'AWI-CM-1-1-LR', 'AWI-CM-1-1-MR', 'AWI-ESM-1-1-LR', 'AWI-ESM-1-REcoM', 'AWI-ESM-2-1-LR', 'BCC-CSM2-HR', 'BCC-CSM2-MR', 'BCC-ESM1', 'CAM-MPAS-HR', 'CAM-MPAS-LR', 'CAMS-CSM1-0', 'CAS-ESM2-0', 'CESM1-1-CAM5-CMIP5', 'CESM1-CAM5-SE-HR', 'CESM1-CAM5-SE-LR', 'CESM1-WACCM-SC', 'CESM2', 'CESM2-FV2', 'CESM2-WACCM', 'CESM2-WACCM-FV2', 'CIESM', 'CMCC-CM2-HR4', 'CMCC-CM2-SR5', 'CMCC-CM2-VHR4', 'CMCC-ESM2', 'CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CNRM-ESM2-1', 'CanESM5', 'CanESM5-1', 'CanESM5-CanOE', 'E3SM-1-0', 'E3SM-1-1', 'E3SM-1-1-ECA', 'E3SM-2-0', 'E3SM-2-0-NARRM', 'E3SM-2-1', 'EC-Earth3', 'EC-Earth3-AerChem', 'EC-Earth3-CC', 'EC-Earth3-GrIS', 'EC-Earth3-HR', 'EC-Earth3-LR', 'EC-Earth3-Veg', 'EC-Earth3-Veg-LR', 'EC-Earth3P', 'EC-Earth3P-HR', 'EC-Earth3P-VHR', 'ECMWF-IFS-HR', 'ECMWF-IFS-LR', 'ECMWF-IFS-MR', 'FGOALS-f3-H', 'FGOALS-f3-L', 'FGOALS-g3', 'FIO-ESM-2-0', 'GFDL-AM4', 'GFDL-CM4', 'G

## extract ocean level info

In [4]:
native = {}
for count, srcId in enumerate(srcIds):
    tmp = srcIds[srcId]["model_component"]["ocean"]["description"]
    tmp = tmp.split(";")
    for val in tmp:
        if "levels" in val:
            #print(srcId, val[:3])
            native[srcId] = int(val[:3])

## search for thetao

In [5]:
%%time
dpaths = xs.findPaths('historical', 'thetao', 'mon', realm='ocean', cmipTable='Omon', deduplicate=True)
#for count, sim in enumerate(dpaths):
#    print("{:03d}".format(count), sim)
#print(dpaths.keys())
sims = xs.getValuesForFacet(dpaths, 'model', 'IPSL-CM5A2-INCA') ## 'E3SM-2-0')  # also E3SM-1-0 (E3SM-Project (1-5), UCSB, E3SM-Project), E3SM-1-1 + ECA, E3SM-2-0-NARRM
sims

CPU times: user 46.7 ms, sys: 4.9 ms, total: 51.6 ms
Wall time: 72.4 ms


['/p/css03/esgf_publish/CMIP6/CMIP/IPSL/IPSL-CM5A2-INCA/historical/r1i1p1f1/Omon/thetao/gn/v20200729/']

In [14]:
%%time
exp = "historical"
var = "thetao"
freq = "mon"
cmipTable = "Omon"
mipEra = "CMIP6"
firstMemberOnly = True

dpaths = xs.findPaths(exp, var, freq, cmipTable=cmipTable, mip_era=mipEra)
models = xs.natural_sort(xs.getGroupValues(dpaths, 'model'))
#print('models:', models)
#print('number of models:', len(models))

badFilesTime = [
    "/p/css03/esgf_publish/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/historical/r1i1p1f1/Omon/thetao/gn/v20200624/thetao_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_185001-185012.nc",
    "/p/css03/esgf_publish/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-CC/historical/r1i1p1f1/Omon/thetao/gn/v20210113/thetao_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_185001-185012.nc",
    "/p/css03/esgf_publish/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/historical/r1i1p1f1/Omon/thetao/gn/v20200919/thetao_Omon_EC-Earth3-Veg-LR_historical_r1i1p1f1_gn_185001-185012.nc",
    "/p/css03/esgf_publish/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg/historical/r1i1p1f1/Omon/thetao/gn/v20211207/thetao_Omon_EC-Earth3-Veg_historical_r1i1p1f1_gn_185001-185012.nc",
    "/p/css03/scratch/cmip6/CMIP/CAS/FGOALS-f3-L/historical/r1i1p1f1/Omon/thetao/gn/v20191007/thetao_Omon_FGOALS-f3-L_historical_r1i1p1f1_gn_185001-189912.nc",
];

for count, model in enumerate(models):
    dpaths_model = xs.retainDataByFacetValue(dpaths, 'model', model)
    members = xs.natural_sort(xs.getGroupValues(dpaths_model, 'member'))
    if firstMemberOnly:
        members = members[0:1]
    #print(count, model, members)
    for member in members:
        dpaths_model_member_list = xs.getValuesForFacet(dpaths_model, 'member', member)
        if len(dpaths_model_member_list) > 1:
            print('Error: multiple paths detected for ', model, member, ': ', dpaths_model_member_list)
        else:
            dpath = dpaths_model_member_list[0]
            ncfile = xs.natural_sort(glob.glob(os.path.join(dpath, '*.nc')))[0]
        # now poll vertical axis
        ds = xc.open_dataset(ncfile, decode_times=False)
        z = xc.get_dim_coords(ds, "Z")
        natZ = int(native[model])
        if natZ < len(z):
            s = "misreported"
        elif len(z) != natZ:
            s = "decimated"

        else:
            s = ""
        print(count, model, members, "output:", len(z), "native:", natZ, s)

0 ACCESS-CM2 ['r1i1p1f1'] output: 50 native: 50 
1 ACCESS-ESM1-5 ['r1i1p1f1'] output: 50 native: 50 
2 AWI-CM-1-1-MR ['r1i1p1f1'] output: 46 native: 46 
3 AWI-ESM-1-1-LR ['r1i1p1f1'] output: 46 native: 46 
4 BCC-CSM2-MR ['r1i1p1f1'] output: 40 native: 40 
5 BCC-ESM1 ['r1i1p1f1'] output: 40 native: 40 
6 CAMS-CSM1-0 ['r1i1p1f1'] output: 50 native: 50 
7 CanESM5 ['r1i1p1f1'] output: 45 native: 45 
8 CanESM5-1 ['r1i1p1f1'] output: 45 native: 45 
9 CanESM5-CanOE ['r1i1p2f1'] output: 45 native: 45 
10 CAS-ESM2-0 ['r1i1p1f1'] output: 30 native: 30 
11 CESM2 ['r1i1p1f1'] output: 33 native: 60 decimated


/home/durack1/mambaforge/envs/xcd061nctax/lib/python3.11/site-packages/xarray/conventions.py:436: SerializationWarning: variable 'thetao' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


12 CESM2-FV2 ['r1i1p1f1'] output: 33 native: 60 decimated
13 CESM2-WACCM ['r1i1p1f1'] output: 33 native: 60 decimated
14 CESM2-WACCM-FV2 ['r1i1p1f1'] output: 33 native: 60 decimated
15 CIESM ['r1i1p1f1'] output: 60 native: 46 misreported
16 CMCC-CM2-HR4 ['r1i1p1f1'] output: 50 native: 50 
17 CMCC-CM2-SR5 ['r1i1p1f1'] output: 50 native: 50 
18 CMCC-ESM2 ['r1i1p1f1'] output: 50 native: 50 
19 CNRM-CM6-1 ['r1i1p1f2'] output: 75 native: 75 
20 CNRM-CM6-1-HR ['r1i1p1f2'] output: 75 native: 75 
21 CNRM-ESM2-1 ['r1i1p1f2'] output: 75 native: 75 
22 E3SM-1-0 ['r1i1p1f1'] output: 60 native: 60 
23 E3SM-1-1 ['r1i1p1f1'] output: 60 native: 60 
24 E3SM-1-1-ECA ['r1i1p1f1'] output: 60 native: 60 
25 E3SM-2-0 ['r1i1p1f1'] output: 60 native: 60 
26 E3SM-2-0-NARRM ['r1i1p1f1'] output: 60 native: 60 
27 EC-Earth3 ['r1i1p1f1'] output: 75 native: 75 
28 EC-Earth3-AerChem ['r1i1p1f1'] output: 75 native: 75 
29 EC-Earth3-CC ['r1i1p1f1'] output: 75 native: 75 
30 EC-Earth3-Veg ['r1i1p1f1'] output: 75 native